In [1]:
import numpy as np # linear algebra
import pathlib
from datetime import datetime, date
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, count, col, format_number, monotonically_increasing_id, hour, mean,minute

Configuração do Spark

In [2]:
import findspark
import os
os.environ["SPARK_HOME"] = "/Users/fabioleite/Downloads/spark-3.5.3-bin-hadoop3"
findspark.init()

In [3]:
spark = SparkSession.builder.appName("Meu App") \
    .config("spark.executor.extraJavaOptions", "-Djava.security.manager=allow") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 22:44:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Leitura dos arquivos

In [4]:
# Caminhos
path_accidents_in_the_Czech_Republic = "Road_accidents_in_the_Czech_Republic/arquivo_base/road_accidents_czechia_2016_2022.csv"
path_pedestrian = "Road_accidents_in_the_Czech_Republic/arquivo_base/pedestrian.csv"
# Leituras e tratamentos iniciais
df_pedestrians = spark.read.csv(path_pedestrian, header=True, sep=',')
df_accidents = spark.read.csv(path_accidents_in_the_Czech_Republic, header=True, sep=',')

In [5]:
df_accidents.describe()
# df.printSchema()
# df.show(truncate=False)

DataFrame[summary: string, id: string, communication_kind: string, date: string, time: string, accident_kind: string, crash_kind: string, fixed_precaution_kind: string, injury: string, cause_of_accident: string, alcohol: string, main_cause_of_accident: string, killed_persons: string, severely_injured_persons: string, slightly_injured_persons: string, total_material_damage: string, road_surface_type: string, condition_of_vehicle_surface: string, condition_of_communication: string, weather_conditions_at_the_time_of_accident: string, visibility: string, visibility_ratios: string, communication_division: string, communication_accident_situation: string, traffic_control_at_the_time_of_an_accident: string, local_administration_priority_in_traffic: string, specific_location_and_objects_in_the_location_of_the_accident: string, directional_rates: string, number_of_vehicles_involved: string, location_of_the_traffic_accident: string, vehicle_type: string, vehicle_brand: string, year_of_manufactur

### 1. Gere um arquivo chamado “alcohol_accidents.csv” a partir do arquivo “road_accidents_czechia_2016_2022.csv”, filtrar os acidentes em que houve detecção do ingerimento de bebida alcoólica. (filtrar pelo campo alcohol)

+ Gere um arquivo chamado “alcohol_accidents.csv”.

In [6]:
# Agrupar por 'alcohol' e contar as ocorrências
df_alcohol_count = df_accidents.groupBy('alcohol').agg(count('*').alias('count'))

# Mostrar o resultado
df_alcohol_count.show(truncate=False)

+----------------------------------------------------------------------------+------+
|alcohol                                                                     |count |
+----------------------------------------------------------------------------+------+
|yes blood alcohol content between 0.5 ‰ and 0.8 ‰                           |1992  |
|ne                                                                          |380032|
|with a driving licence of category B                                        |1304  |
|not detected                                                                |287516|
|with a driving licence of category D                                        |19    |
|NULL                                                                        |6785  |
|Rejected                                                                    |1847  |
|without a driving licence                                                   |78    |
|yes blood alcohol content between 0.24 ‰ and 0.5 ‰   

In [7]:
# Leitura e tratamento dos arquivos do aluno
path_alcohol_accidents_aluno = "Road_accidents_in_the_Czech_Republic/transformações/alcohol_accidents.csv"
df_alcohol_accidents_aluno = spark.read.csv(path_alcohol_accidents_aluno, header=True, sep=',')

In [8]:
# prompt: preciso selecionar todas as ocorrências de registros do df_accidents que iniciam com o valor 'yes blood alcohol content' na coluna alcohol

# Selecionar todas as ocorrências de registros que iniciam com 'yes blood alcohol content' na coluna 'alcohol'
df_accidents_alcohol = df_accidents.filter(df_accidents.alcohol.startswith('yes blood alcohol content'))

# Mostrar o resultado
df_accidents_alcohol.show()

24/11/25 22:45:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+------------------+----------+------+--------------------+--------------------+---------------------+------+--------------------+--------------------+----------------------+--------------+------------------------+------------------------+---------------------+-----------------+----------------------------+--------------------------+------------------------------------------+--------------------+-----------------+----------------------+--------------------------------+------------------------------------------+----------------------------------------+-------------------------------------------------------------+--------------------+---------------------------+--------------------------------+--------------------+-------------+-------------------+--------------------+----+--------------------+--------------------------------+-----------------------------------------+---------------------+----------------------------------------+--------------------+--------------------+--

In [9]:
df_accidents_ne_filtered = df_accidents.filter(df_accidents.alcohol.startswith('yes blood alcohol content'))
path_alcohol_accidents = "Road_accidents_in_the_Czech_Republic/transformações/alcohol_accidents.csv"
df_alcohol_accidents = spark.read.csv(path_alcohol_accidents, header=True, sep=',')


In [10]:
df_alcohol_accidents.describe('alcohol').show()

+-------+--------------------+
|summary|             alcohol|
+-------+--------------------+
|  count|               30848|
|   mean|                NULL|
| stddev|                NULL|
|    min|yes blood alcohol...|
|    max|yes blood alcohol...|
+-------+--------------------+



In [11]:
df_accidents_ne_filtered.describe('alcohol').show()

+-------+--------------------+
|summary|             alcohol|
+-------+--------------------+
|  count|               30848|
|   mean|                NULL|
| stddev|                NULL|
|    min|yes blood alcohol...|
|    max|yes blood alcohol...|
+-------+--------------------+



In [12]:

# Comparar os dois DataFrames usando um inner join
df_comparison = df_alcohol_accidents.join(df_accidents_ne_filtered, on=['alcohol'], how='inner')

# Ou, se você quiser comparar apenas a coluna 'alcohol', pode usar algo como:
df_alcohol_accidents_alcohol_column = df_alcohol_accidents.select('alcohol')
df_accidents_ne_filtered_alcohol_column = df_accidents_ne_filtered.select('alcohol')
# Encontrar a diferença (valores únicos em cada DataFrame)
df_difference = df_alcohol_accidents_alcohol_column.subtract(df_accidents_ne_filtered_alcohol_column)
df_difference.show()

+-------+
|alcohol|
+-------+
+-------+



In [13]:
# prompt: preciso exibir o total de registros do dataframe df_alcohol_accidents e df_accidents_ne_filtered

print("Total de registros em df_alcohol_accidents:", df_alcohol_accidents.count())
print("Total de registros em df_accidents_ne_filtered:", df_accidents_ne_filtered.count())

Total de registros em df_alcohol_accidents: 30848


Total de registros em df_accidents_ne_filtered: 30848


### 2. Gere um arquivo chamado ”accidents_NCBMV.csv”, filtre pelo campo “crash_kind” os acidentes em que não houve colisão entre veículos em movimento. (filtrando por: not an option It is not a collision between moving vehicles)

+ Gere um arquivo chamado “accidents_NCBMV.csv”.

In [14]:
df_accidents_filtered_crash_kind = df_accidents.filter(df_accidents.crash_kind == 'not an option It is not a collision between moving vehicles')
df_accidents_filtered_crash_kind.show()

+--------------+------------------+----------+------+--------------------+--------------------+---------------------+------+--------------------+------------+----------------------+--------------+------------------------+------------------------+---------------------+-----------------+----------------------------+--------------------------+------------------------------------------+--------------------+-----------------+----------------------+--------------------------------+------------------------------------------+----------------------------------------+-------------------------------------------------------------+--------------------+---------------------------+--------------------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------------------+-----------------------------------------+---------------------+----------------------------------------+--------------------+--------

In [15]:
df_accidents_filtered_crash_kind.describe('crash_kind').show()

+-------+--------------------+
|summary|          crash_kind|
+-------+--------------------+
|  count|              471684|
|   mean|                NULL|
| stddev|                NULL|
|    min|not an option It ...|
|    max|not an option It ...|
+-------+--------------------+



In [16]:
path_accidents_NCBMV = 'Road_accidents_in_the_Czech_Republic/transformações/accidents_NCBMV.csv'
df_accidents_NCBMV = spark.read.csv(path_accidents_NCBMV, header=True, sep=',')

In [17]:
df_accidents_NCBMV.describe('crash_kind').show()

+-------+--------------------+
|summary|          crash_kind|
+-------+--------------------+
|  count|              471684|
|   mean|                NULL|
| stddev|                NULL|
|    min|not an option It ...|
|    max|not an option It ...|
+-------+--------------------+



In [18]:
# Comparar os dois DataFrames usando um inner join
df_comparison = df_accidents_filtered_crash_kind.join(df_accidents_NCBMV, on=['crash_kind'], how='inner')

# Ou, se você quiser comparar apenas a coluna 'alcohol', pode usar algo como:
df_accidents_filtered_crash_kind_column = df_accidents_filtered_crash_kind.select('id','crash_kind')
df_accidents_NCBMV_column = df_accidents_NCBMV.select('id','crash_kind')
# Encontrar a diferença (valores únicos em cada DataFrame)
df_difference = df_accidents_filtered_crash_kind_column.subtract(df_accidents_NCBMV_column)
df_difference.count()

0

In [19]:
df_accidents_NCBMV.show()

+--------------+------------------+----------+------+--------------------+--------------------+---------------------+------+--------------------+------------+----------------------+--------------+------------------------+------------------------+---------------------+-----------------+----------------------------+--------------------------+------------------------------------------+--------------------+-----------------+----------------------+--------------------------------+------------------------------------------+----------------------------------------+-------------------------------------------------------------+--------------------+---------------------------+--------------------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------------------+-----------------------------------------+---------------------+----------------------------------------+--------------------+--------

In [20]:
df_difference2 = df_accidents_NCBMV_column.subtract(df_accidents_filtered_crash_kind_column)
df_difference2.show(truncate=False)

+---+----------+
|id |crash_kind|
+---+----------+
+---+----------+



In [21]:
print("Total de registros em df_accidents_filtered_crash_kind:", df_accidents_filtered_crash_kind.count())
print("Total de registros em df_accidents_NCBMV:", df_accidents_NCBMV.count())

Total de registros em df_accidents_filtered_crash_kind: 471684
Total de registros em df_accidents_NCBMV: 471684


### 3. Considere o arquivo gerado no item 2 (accidents_NCBMV.csv), filtre pelo campo “accident_kind” os acidentes causados por animais da floresta. (collision with forest animals)
+ Gere um arquivo chamado “accidents_CWFA.csv”.

In [22]:
path_accidents_NCBMV_animals = 'Road_accidents_in_the_Czech_Republic/transformações/accidents_NCBMV.csv'
df_accidents_NCBMV = spark.read.csv(path_accidents_NCBMV, header=True, sep=',')

In [23]:
df_accidents_animals = df_accidents.filter(df_accidents.accident_kind == 'collision with forest animals')
df_accidents_animals.describe('accident_kind').show()

+-------+--------------------+
|summary|       accident_kind|
+-------+--------------------+
|  count|               92632|
|   mean|                NULL|
| stddev|                NULL|
|    min|collision with fo...|
|    max|collision with fo...|
+-------+--------------------+



In [24]:
df_accidents_NCBMV.describe('accident_kind').show()

+-------+--------------------+
|summary|       accident_kind|
+-------+--------------------+
|  count|              471684|
|   mean|                NULL|
| stddev|                NULL|
|    min|another type of a...|
|    max|             crashes|
+-------+--------------------+



In [26]:
# Ou, se você quiser comparar apenas a coluna 'alcohol', pode usar algo como:
df_accidents_animals_column = df_accidents_animals.select('id', 'accident_kind')
df_accidents_NCBMV_column = df_accidents_NCBMV.select('id', 'accident_kind')
# Encontrar a diferença (valores únicos em cada DataFrame)
df_difference = df_accidents_animals_column.subtract(df_accidents_NCBMV_column)

print("Total de registros em df_accidents_animals:", df_accidents_animals.count())
print("Total de registros em df_accidents_NCBMV:", df_accidents_NCBMV.count())
print("Total de registros que df_accidents_animals_column possui a mais:", df_difference.count())

Total de registros em df_accidents_animals: 92632
Total de registros em df_accidents_NCBMV: 471684


Total de registros que df_accidents_animals_column possui a mais: 0


### 4. Filtre pela coluna “pedestrian_category.csv” no arquivo “pedestre.csv” os pedestres que estavam sob a influência de álcool.
+ Gere um arquivo chamado “drunk_pedestrians.csv”

In [27]:
path_drunk_pedestrians = 'Road_accidents_in_the_Czech_Republic/transformações/drunk_pedestrians.csv'
df_accidents_drunk_pedestrians = spark.read.csv(path_drunk_pedestrians, header=True, sep=',')
# df_accidents_drunk_pedestrians.show(truncate=False)
df_drunk_pedestrians_grp= df_accidents_drunk_pedestrians.groupBy('pedestrian_category').agg(count('*').alias('count'))
df_drunk_pedestrians_grp.show(truncate=False)

+----------------------+-----+
|pedestrian_category   |count|
+----------------------+-----+
|man                   |1842 |
|child (up to 15 years)|5    |
|Woman                 |399  |
|another group         |22   |
+----------------------+-----+



In [28]:
df_pedestrians.show(truncate=False)

+----------+----------+----------------------+-------------------------------------------------------------------------+---------------------------------------------------------+------------------------------------------------------------------------+
|Unnamed: 0|id        |pedestrian_category   |pedestrian_condition                                                     |pedestrian_behaviour                                     |situation_at_the_point_of_accident                                      |
+----------+----------+----------------------+-------------------------------------------------------------------------+---------------------------------------------------------+------------------------------------------------------------------------+
|0         |2100180003|Woman                 |under the influence of alcohol with a blood alcohol content of 1‰ or more|sudden entry into the roadway from the sidewalk, shoulder|another situation                                                 

In [29]:
df_pedestrians.groupBy('pedestrian_condition').agg(count('*').alias('count')).show(truncate=False)

+---------------------------------------------------------------------------+-----+
|pedestrian_condition                                                       |count|
+---------------------------------------------------------------------------+-----+
|not detected                                                               |2355 |
|other condition not specified                                              |111  |
|inattention, distraction                                                   |2738 |
|suicide attempt, suicide                                                   |36   |
|invalid                                                                    |70   |
|good no adverse circumstances were found                                   |15181|
|physical disability (illness, nausea, reduced mobility, etc.)              |134  |
|under the influence of alcohol with a blood alcohol content of 1‰ or more  |1758 |
|under the influence of drugs, narcotics                                    

In [30]:
# prompt: preciso selecionar os registros do df_pedestrians que possuem como valor para o campo pedestrian_condition começando com a string under the influence of alcohol

# Filtrar os pedestres que estavam sob a influência de álcool
df_drunk_pedestrians = df_pedestrians.filter(df_pedestrians.pedestrian_condition.startswith('under the influence of alcohol'))

# Mostrar o resultado
df_drunk_pedestrians.show()

# Salvar o resultado em um arquivo CSV (opcional)
# path_drunk_pedestrians = '/content/drive/MyDrive/UEPB_Drive/2024.2/LEDA/Accidents_Czech/transformações/drunk_pedestrians.csv'
# df_drunk_pedestrians.write.csv(path_drunk_pedestrians, header=True)

+----------+----------+-------------------+--------------------+--------------------+----------------------------------+
|Unnamed: 0|        id|pedestrian_category|pedestrian_condition|pedestrian_behaviour|situation_at_the_point_of_accident|
+----------+----------+-------------------+--------------------+--------------------+----------------------------------+
|         0|2100180003|              Woman|under the influen...|sudden entry into...|                 another situation|
|         2|2100180039|                man|under the influen...|sudden entry into...|                 another situation|
|        51|2100181282|                man|under the influen...|poor estimation o...|                 another situation|
|        64|2100181445|              Woman|under the influen...|correct, appropriate|              crossing at a mar...|
|        66|2100181510|                man|under the influen...|poor estimation o...|              Crossing outside ...|
|        77|2100181889|         

In [31]:
# prompt: preciso comparar dois dataframes df_accidents_drunk_pedestrians e df_drunk_pedestrians mostrar as diferenças e imprimir o total de registros de cada um

# Comparar os dois DataFrames usando um inner join
df_comparison = df_accidents_drunk_pedestrians.join(df_drunk_pedestrians, on=['pedestrian_category'], how='inner')

# Ou, se você quiser comparar apenas a coluna 'pedestrian_category', pode usar algo como:
df_accidents_drunk_pedestrians_column = df_accidents_drunk_pedestrians.select('pedestrian_category')
df_drunk_pedestrians_column = df_drunk_pedestrians.select('pedestrian_category')

# Encontrar a diferença (valores únicos em cada DataFrame)
df_difference = df_accidents_drunk_pedestrians_column.subtract(df_drunk_pedestrians_column)
df_difference.show()

print("Total de registros em df_accidents_drunk_pedestrians:", df_accidents_drunk_pedestrians.count())
print("Total de registros em df_drunk_pedestrians:", df_drunk_pedestrians.count())

+-------------------+
|pedestrian_category|
+-------------------+
+-------------------+

Total de registros em df_accidents_drunk_pedestrians: 2268
Total de registros em df_drunk_pedestrians: 2268


## Ordenações
+ Ordenar o arquivo pela data do acidente (campo date) em ordem decrescente, da data mais recente para a menos recente.
+  Ordenar o arquivo accidents_NCBMV.csv pela coluna “time” (campo time) em ordem crescente, do menor para o maior.
+ Ordenar o arquivo accidents_NCBMV.csv pelo tipo de comunicação (campo communication_kind), em ordem alfabética crescente.